![title](p2.jpg)

Welcome to the "M5 Forecasting - Accuracy" competition! In this competition, contestants are challenged to forecast future sales at Walmart based on heirarchical sales in the states of California, Texas, and Wisconsin. 

### The Goal

The objective of the task is to find he **most accurate point forecasts** for each of the 42,840 time series of the competition. 

The time series are shown below -

![title](p3.png)

# The dataset <a id="1"></a>

The M5 dataset, generously made available by Walmart, involves the unit sales of various products sold in the USA, organized in the form of grouped time series. More specifically, the dataset involves the unit sales of 3,049 products, classified in 3 product categories (Hobbies, Foods, and Household) and 7 product departments, in which the above-mentioned categories are disaggregated.  The products are sold across ten stores, located in three States (CA, TX, and WI).


The dataset consists of five .csv files.

* <code>calendar.csv</code> - Contains the dates on which products are sold. The dates are in a <code>yyyy/dd/mm</code> format.

* <code>sales_train_validation.csv</code> - Contains the historical daily unit sales data per product and store <code>[d_1 - d_1913]</code>.

* <code>submission.csv</code> - Demonstrates the correct format for submission to the competition.

* <code>sell_prices.csv</code> - Contains information about the price of the products sold per store and date.

* <code>sales_train_evaluation.csv</code> - Available one month before the competition deadline. It will include sales for <code>[d_1 - d_1941]</code>.

In this competition, we need to forecast the sales for <code>[d_1942 - d_1969]</code>. These rows form the evaluation set. The rows <code>[d_1914 - d_1941]</code> form the validation set, and the remaining rows form the training set. Now, since we understand the dataset and know what to predict, let us visualize the dataset.

![title](p1.png)

## EDA

In [2]:
import os
import gc
import time
import math
import datetime
from math import log, floor
from sklearn.neighbors import KDTree


import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.utils import shuffle
#from tqdm.notebook import tqdm as tqdm


import scipy
import statsmodels
from scipy import signal
import statsmodels.api as sm
from fbprophet import Prophet
from scipy.signal import butter, deconvolve
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt


import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import pywt
from statsmodels.robust import mad


import seaborn as sns
from matplotlib import colors
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize


import warnings
warnings.filterwarnings("ignore")

In [3]:
INPUT_DIR = '../input/m5-forecasting-accuracy'
calendar = pd.read_csv(f'{INPUT_DIR}/calendar.csv')
selling_prices = pd.read_csv(f'{INPUT_DIR}/sell_prices.csv')
sample_submission = pd.read_csv(f'{INPUT_DIR}/sample_submission.csv')
sales_train_val = pd.read_csv(f'{INPUT_DIR}/sales_train_validation.csv')

We consider 3 time series here, that is the unit sale of item X in a specific store, here we choose:

1. FOODS_3_388_CA_1_validation
2. HOBBIES_2_148_CA_1_validation
3. HOUSEHOLD_2_468_CA_1_validation

Corresponding to id 10k,20k and 30k respectively.

In [4]:
ids = sorted(list(set(sales_train_val['id'])))
d_cols = [c for c in sales_train_val.columns if 'd_' in c]
x_1 = sales_train_val.loc[sales_train_val['id'] == ids[10000]].set_index('id')[d_cols].values[0]
x_2 = sales_train_val.loc[sales_train_val['id'] == ids[20000]].set_index('id')[d_cols].values[0]
x_3 = sales_train_val.loc[sales_train_val['id'] == ids[30000]].set_index('id')[d_cols].values[0]


#### Visualising a part of Sales data

In [ ]:
fig = make_subplots(rows=3, cols=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_1)), y=x_1,),
             row=1, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_2)), y=x_2,),
             row=2, col=1)

fig.add_trace(go.Scatter(x=np.arange(len(x_3)), y=x_3,),
             row=3, col=1)

fig.update_layout(height=1200, width=800, title_text="Sample sales")
fig.show()

#### Smoothing

In [ ]:
def average_smoothing(signal, kernel_size=3, stride=1):
    sample = []
    start = 0
    end = kernel_size
    while end <= len(signal):
        start = start + stride
        end = end + stride
        sample.extend(np.ones(end - start)*np.mean(signal[start:end]))
    return np.array(sample)

In [ ]:
y_a1 = average_smoothing(x_1)
y_a2 = average_smoothing(x_2)
y_a3 = average_smoothing(x_3)

fig = make_subplots(rows=3, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_1)), mode='lines+markers', y=x_1, marker=dict(color="lightskyblue"), showlegend=False,
               name="Original sales"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_1)), y=y_a1, mode='lines', marker=dict(color="navy"), showlegend=False,
               name="Denoised sales"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_2)), mode='lines+markers', y=x_2, marker=dict(color="thistle"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_2)), y=y_a2, mode='lines', marker=dict(color="indigo"), showlegend=False),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_3)), mode='lines+markers', y=x_3, marker=dict(color="mediumaquamarine"), showlegend=False),
    row=3, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(len(x_3)), y=y_a3, mode='lines', marker=dict(color="darkgreen"), showlegend=False),
    row=3, col=1
)

fig.update_layout(height=1200, width=800, title_text="Original (pale) vs. Denoised (dark) signals")
fig.show()

### Rolling Average Sales with Time

In [ ]:
past_sales = sales_train_val.set_index('id')[d_cols] \
    .T \
    .merge(calendar.set_index('d')['date'],
           left_index=True,
           right_index=True,
            validate='1:1') \
    .set_index('date')

store_list = selling_prices['store_id'].unique()
means = []
fig = go.Figure()
for s in store_list:
    store_items = [c for c in past_sales.columns if s in c]
    data = past_sales[store_items].sum(axis=1).rolling(90).mean()
    means.append(np.mean(past_sales[store_items].sum(axis=1)))
    fig.add_trace(go.Scatter(x=np.arange(len(data)), y=data, name=s))
    
fig.update_layout(yaxis_title="Sales", xaxis_title="Time", title="Rolling Average Sales vs. Time (per store)")

### Modeling a part of data



In [ ]:
!pip install pmdarima
!pip install sktime


In [ ]:
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting import smape_loss


In [ ]:
Overall = sales_train_val.sum(axis=0,skipna=True,numeric_only=True)


In [ ]:
Overall1 = pd.Series(pd.Series.to_numpy(Overall)[-500:])

In [ ]:
y_train, y_test = temporal_train_test_split(Overall1, test_size=50)
print(y_train.shape[0], y_test.shape[0])


In [ ]:
fh = np.arange(50)+1

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(450), mode='lines', y=y_train.values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Original signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_test.values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)


## Naive approach <a id="3.2"></a>

It simply forecasts the next day's sales as the current day's sales. The model can be summarized as follows:

<img src="https://i.imgur.com/r8wjrzk.png" width="110px">


In [ ]:
forecaster = NaiveForecaster(strategy="last")
forecaster.fit(y_train)
y_last = forecaster.predict(fh)

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(450), mode='lines', y=y_train.values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Original signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_test.values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_last.values, mode='lines', marker=dict(color="green"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)


##### Dynamic Naive

In [ ]:
forecaster = NaiveForecaster(strategy="last")
forecaster.fit(y_train)
from sktime.forecasting.model_selection import SlidingWindowSplitter
cv = SlidingWindowSplitter(fh=1)
y_pred = forecaster.update_predict(y_test, cv)

In [ ]:
fig = make_subplots(rows=1, cols=1)


fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_test.values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_pred.values, mode='lines', marker=dict(color="green"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)


## Exponential smoothing <a id="3.5"></a>

The **exponential smoothing** method uses a different type of smoothing which differs from average smoothing. The previous time steps are exponentially weighted and added up to generate the forecast. The weights decay as we move further backwards in time. The model can be summarized as follows:

<img src="https://i.imgur.com/IqqjOFc.png" width="520px">
<img src="https://i.imgur.com/GiyHyZf.png" width="255px">

In the above equations, $\alpha$ is the smoothing parameter. The forecast *y<sub>t+1</sub>* is a weighted average of all the observations in the series *y<sub>1</sub>, … ,y<sub>t</sub>*. The rate at which the weights decay is controlled by the parameter *α*. This method gives different weightage to different time steps, instead of giving the same weightage to all time steps (like the moving average method). This ensures that recent sales data is given more importance than old sales data while making the forecast. Now let us see how this new smoothing method performs on our miniature dataset. The training data is in <font color="blue">blue</font>, validation data in <font color="darkorange">orange</font>, and predictions in <font color="green">green</font>.

In [ ]:
from sktime.forecasting.exp_smoothing import ExponentialSmoothing
forecaster = ExponentialSmoothing(trend="add", seasonal="multiplicative", sp=14)
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
smape_loss(y_test, y_pred)

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(300,450), mode='lines', y=y_train.values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Original signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_test.values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_pred.values, mode='lines', marker=dict(color="green"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)


## ARIMA <a id="3.6"></a>

**ARIMA** stands for **A**uto **R**egressive **I**ntegrated **M**oving **A**verage. While exponential smoothing models were based on a description of trend and seasonality in data, ARIMA models aim to describe the correlations in the time series. 

In [ ]:
from sktime.forecasting.arima import AutoARIMA
forecaster = AutoARIMA(sp=14, suppress_warnings=True)
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)


In [ ]:
smape_loss(y_test, y_pred)

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(300,450), mode='lines', y=y_train.values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Original signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_test.values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_pred.values, mode='lines', marker=dict(color="green"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)



### Ensembling
 we can combine different variants of exponential smoothing as follows:

In [ ]:
from sktime.forecasting.compose import EnsembleForecaster
forecaster = EnsembleForecaster([
    ("ses", ExponentialSmoothing(seasonal="multiplicative", sp=14)),
    ("holt", ExponentialSmoothing(trend="add", damped=False, seasonal="multiplicative", sp=14)),
    ("damped", ExponentialSmoothing(trend="add", damped=True, seasonal="multiplicative", sp=14))
])
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
smape_loss(y_test, y_pred)

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(300,450), mode='lines', y=y_train.values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Original signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_test.values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_pred.values, mode='lines', marker=dict(color="green"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)


### Detrending

We can try to see if the above time series has any long term trends. Let's check for linear trend-

We can see below that the trend is small incrementing.

In [ ]:
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.transformers.single_series.detrend import Detrender

# liner detrending
forecaster = PolynomialTrendForecaster(degree=1)
transformer = Detrender(forecaster=forecaster)
yt = transformer.fit_transform(y_train)

# internally, the Detrender uses the in-sample predictions of the PolynomialTrendForecaster
forecaster = PolynomialTrendForecaster(degree=1)
fh_ins = -np.arange(len(y_train)) # in-sample forecasting horizon
y_pred = forecaster.fit(y_train).predict(fh=fh_ins)


In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(450), mode='lines', y=y_train.values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Original signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange( 500), y=y_pred.values, mode='lines', marker=dict(color="green"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)


### Pipelining

Let's use the detrender in a pipeline together with de-seasonalisation. And finally a regressor.

In [ ]:
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformers.single_series.detrend import Deseasonalizer
from sktime.forecasting.compose import ReducedRegressionForecaster
from sklearn.neighbors import KNeighborsRegressor

regressor = KNeighborsRegressor(n_neighbors=1)
forecaster = TransformedTargetForecaster([
    ("deseasonalise", Deseasonalizer(model="multiplicative", sp=12)),
    ("detrend", Detrender(forecaster=PolynomialTrendForecaster(degree=1))),
    ("forecast", ReducedRegressionForecaster(regressor=regressor, window_length=15, strategy="recursive"))
])
forecaster.fit(y_train)
y_pred = forecaster.predict(fh)
smape_loss(y_test, y_pred)

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Scatter(x=np.arange(300,450), mode='lines', y=y_train.values, marker=dict(color="dodgerblue"), showlegend=False,
               name="Original signal"),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_test.values, mode='lines', marker=dict(color="darkorange"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)
fig.add_trace(
    go.Scatter(x=np.arange(450, 500), y=y_pred.values, mode='lines', marker=dict(color="green"), showlegend=False,
               name="Denoised signal"),
    row=1, col=1
)
